In [65]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [66]:
!pip install -q langchain-groq langchain-core requests

In [67]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [68]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [69]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/dbada9e1e947f8072b38f5d4/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [70]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'}}

In [71]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'PKR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1769644802,
 'time_last_update_utc': 'Thu, 29 Jan 2026 00:00:02 +0000',
 'time_next_update_unix': 1769731202,
 'time_next_update_utc': 'Fri, 30 Jan 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 279.9234}

In [72]:
convert.invoke({'base_currency_value':10, 'conversion_rate':280})

2800

In [73]:
# tool binding
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [74]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [75]:
messages = [HumanMessage('What is the conversion factor between USD and PKR, and based on that can you convert 10 USDr to PKR')]

In [76]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 10 USDr to PKR', additional_kwargs={}, response_metadata={})]

In [77]:
ai_message = llm_with_tools.invoke(messages)

In [78]:
messages.append(ai_message)

In [79]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'PKR'},
  'id': 'g5htgkcc5',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': '9ha689nn9',
  'type': 'tool_call'}]

In [80]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [81]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 10 USDr to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'g5htgkcc5', 'function': {'arguments': '{"base_currency":"USD","target_currency":"PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': '9ha689nn9', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 350, 'total_tokens': 388, 'completion_time': 0.118480567, 'completion_tokens_details': None, 'prompt_time': 0.057246646, 'prompt_tokens_details': None, 'queue_time': 0.057913844, 'total_time': 0.175727213}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c09e6-5d25-77e3-9e

In [82]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and PKR is 279.9234. Based on this conversion factor, 10 USD is equivalent to 2799.234 PKR.'